#### Tasks under this project: 
- We're going to scrape Github topic url: 
- From list of topics. To Scrape Topic Title, Topic Description, Topic page URL
- For each topic we'll get the top 25 repos.
- For each topic, we'll get the repo name, username, stars & url.
- Stage1: For each topic we'll create a csv file in format
- Csv1: Topic title,Topic desc., Topic page url Top25
- Csv2: Topics Repo name, User name, Stars, repo URL
- Connecting script to MySQL database & load data on specific tables
#### Overview of tasks: we'll Extract(scrape) + Transform + Load Data(connect to a Database)

In [1]:
import requests
from bs4 import BeautifulSoup # importing beautifulsoup from bs4 module

In [2]:
topics_url= 'https://github.com/topics'

### Requests() library to crawl the webpage

In [3]:
response= requests.get(topics_url)
response.status_code 

200

In [4]:
# Content of page response.text
content= len(response.text)
content

166882

In [9]:
# pip install html5lib

### Parsing the webpage

In [5]:
# creating a soup using bs4
soup= BeautifulSoup(response.content,'html.parser')
print(soup)


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-b92e9647318f.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/

### now that we've all content Github topics page.. we make our path to target:
     

In [6]:
#Fetching Topics_title name under href topic links

topic_titles = soup.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})
#print(topic_titles)

Topic_title=[]
for i in topic_titles:
    t= i.text.strip()
    Topic_title.append(t)
print(Topic_title)

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android', 'Angular', 'Ansible', 'API', 'Arduino', 'ASP.NET', 'Atom', 'Awesome Lists', 'Amazon Web Services', 'Azure', 'Babel', 'Bash', 'Bitcoin', 'Bootstrap', 'Bot', 'C', 'Chrome', 'Chrome extension', 'Command line interface', 'Clojure', 'Code quality', 'Code review', 'Compiler', 'Continuous integration', 'COVID-19', 'C++']


In [7]:
# Fetching topic_description
topic_des= soup.find_all('p', class_= 'f5 color-fg-muted mb-0 mt-1')
#print(topic_des)

Topic_desc=[]
for i in topic_des:
        x= i.text.strip()
        Topic_desc.append(x)
print(Topic_desc)

['3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.', 'Ajax is a technique for creating interactive web applications.', 'Algorithms are self-contained sequences that carry out a variety of tasks.', 'Amp is a non-blocking concurrency library for PHP.', 'Android is an operating system built by Google designed for mobile devices.', 'Angular is an open source web application platform.', 'Ansible is a simple and powerful automation engine.', 'An API (Application Programming Interface) is a collection of protocols and subroutines for building software.', 'Arduino is an open source platform for building electronic devices.', 'ASP.NET is a web framework for building modern web apps and services.', 'Atom is a open source text editor built with web technologies.', 'An awesome list is a list of awesome things curated by the community.', 'Amazon Web Services provides on-demand cloud computing platforms on a subscription basis.', 'Azure is a cloud co

In [8]:
# Fetching topic Link
topic_links = soup.find_all('a', {'class': "no-underline flex-1 d-flex flex-column"})
#print(topic_titles)

base_url= 'https://github.com/topics'
Topic_link=[]

for i in topic_links:
    L= i.get('href')
    
    Topic_link.append(base_url + L[7:])
    

In [28]:
# topic_links is a soup/bs4 elemt: containing <p> tags elemt + <a> href elemt
topic_links[1]['href']

'/topics/ajax'

In [12]:
L[7:]

'/cpp'

In [29]:
Topic_link

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/atom',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compil

#### Stage 1: Creating a csv file

In [9]:
# Dictionary element

topic_dict= {'Topic_title': Topic_title, 'Topic_desc': Topic_desc, 'Topic_link': Topic_link}
#topic_dict

In [31]:
import pandas as pd

In [16]:
topic= pd.DataFrame(topic_dict, index= None)

topic.to_csv('C:/Users/hp/Web Scraping & APIs/WebScraping/Github-Scraping/github_scrap.csv', index= None)


### Fetching Repos from individual Topics

In [32]:
# requesting multiple topic links(Top 20) 
import requests
from bs4 import BeautifulSoup

header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}

In [33]:
resp= requests.get('https://github.com/topics/3d', headers= header)

In [34]:
resp.status_code

200

In [35]:
soupelemt= BeautifulSoup(resp.content, 'html.parser')

In [40]:
soupelemt


<!DOCTYPE html>

<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-b92e9647318f.css" media="all" rel="stylesheet"><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-5d486a4ede8e.css" media="all" rel="stylesheet"><link crossorigin="anonymous" data-color-theme="dark_dimmed" data-href="https://github.githubassets.com/

In [39]:
type(soupelemt)

bs4.BeautifulSoup

In [43]:
# Underthis <h3>tag & given Class are all the Username+Reponame
Repos= soupelemt.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})
Repos

[<h3 class="f3 color-fg-muted text-normal lh-condensed">
 <a class="Link" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":null,"record_id":97088,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="4bdbc49d3c05ae7f70b531fbce709a384200b0768554e0172950286a8db30940" data-turbo="false" data-view-component="true" href="/mrdoob">
             mrdoob
 </a>          /
           <a class="Link text-bold wb-break-word" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":576201,"originating_url":"https://github.com/topics/3d","user_id":null}}' data-hydro-click-hmac="517d3d5cb9d89752156923904a4238816bc9b51ab7772f3e3644ce897d8dd4e5" data-turbo="false" data-view-component="true"

#### Username & Reponame

In [44]:
def Get_ur(Repos):
    # Underthis <h3>tag & given Class are all the Username+Reponame
    h3_tag = soupelemt.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})
    user_name= []
    repo_name= []

    for repo in Repos:
        a= repo.find_all('a')

        ur_name= a[0].text.strip()
        rep_name= a[1].text.strip()
        user_name.append(ur_name)
        repo_name.append(rep_name)
    print(user_name)

In [45]:
Get_ur(Repos)

['mrdoob', 'pmndrs', 'libgdx', 'BabylonJS', 'ssloy', 'lettier', 'aframevr', 'FreeCAD', 'CesiumGS', 'MonoGame', 'blender', 'metafizzy', 'isl-org', 'timzhang642', 'a1studmuffin', 'nerfstudio-project', 'domlysz', 'FyroxEngine', 'google', 'openscad']


#### Startag

In [36]:
# Now we need Stars of a paritcular repo

def star_count(soupelemt):
    star_tag= soupelemt.find_all('span', {'class': 'Counter js-social-count'})

    stars= []
    for star in star_tag:

        st= star.text
        if st[-1] == 'k':
            st= int(float(st[:-1]) * 1000)
            stars.append(st)
    print(stars)

In [37]:
star_count(soupelemt)

[95600, 24400, 22200, 21600, 18300, 16300, 15800, 15700, 11200, 10300, 10100, 10000, 9700, 9200, 7500, 7000, 6800, 6700, 6100, 6000]


#### Repos links

In [38]:

def repo_link(soupelemt):
    
    repo_linktag= soupelemt.find_all('a', {'class' : 'Link text-bold wb-break-word'})

    Repo_link= []
    l= 'https://github.com/topics'
    for i in repo_linktag:
        links= l + i.get('href')
        Repo_link.append(links)
    print(Repo_link)

In [39]:
repo_link(soupelemt)

['https://github.com/topics/mrdoob/three.js', 'https://github.com/topics/pmndrs/react-three-fiber', 'https://github.com/topics/libgdx/libgdx', 'https://github.com/topics/BabylonJS/Babylon.js', 'https://github.com/topics/ssloy/tinyrenderer', 'https://github.com/topics/lettier/3d-game-shaders-for-beginners', 'https://github.com/topics/aframevr/aframe', 'https://github.com/topics/FreeCAD/FreeCAD', 'https://github.com/topics/CesiumGS/cesium', 'https://github.com/topics/MonoGame/MonoGame', 'https://github.com/topics/metafizzy/zdog', 'https://github.com/topics/blender/blender', 'https://github.com/topics/isl-org/Open3D', 'https://github.com/topics/timzhang642/3D-Machine-Learning', 'https://github.com/topics/a1studmuffin/SpaceshipGenerator', 'https://github.com/topics/nerfstudio-project/nerfstudio', 'https://github.com/topics/domlysz/BlenderGIS', 'https://github.com/topics/FyroxEngine/Fyrox', 'https://github.com/topics/google/model-viewer', 'https://github.com/topics/openscad/openscad']


#### Converting to a CSV

Repo_dict= {'User_name': user_name, 'Repo_name': repo_name, 'Stars': stars, 'Repo_link': Repo_link}

In [40]:
import pandas as pd

In [41]:
len(stars)

20

In [42]:
repo_csv= pd.DataFrame(Repo_dict, index= None)
repo_csv

,User_name,Repo_name,Stars,Repo_link
0,mrdoob,three.js,95.5k,https://github.com/topics/mrdoob/three.js
1,pmndrs,react-three-fiber,24.4k,https://github.com/topics/pmndrs/react-three-f...
2,libgdx,libgdx,22.2k,https://github.com/topics/libgdx/libgdx
3,BabylonJS,Babylon.js,21.6k,https://github.com/topics/BabylonJS/Babylon.js
4,ssloy,tinyrenderer,18.2k,https://github.com/topics/ssloy/tinyrenderer
5,lettier,3d-game-shaders-for-beginners,16.3k,https://github.com/topics/lettier/3d-game-shad...
6,aframevr,aframe,15.8k,https://github.com/topics/aframevr/aframe
7,FreeCAD,FreeCAD,15.6k,https://github.com/topics/FreeCAD/FreeCAD
8,CesiumGS,cesium,11.2k,https://github.com/topics/CesiumGS/cesium
9,MonoGame,MonoGame,10.2k,https://github.com/topics/MonoGame/MonoGame


In [142]:
repo_csv.to_csv('C:/Users/hp/Web Scraping & APIs/WebScraping/Github-Scraping/Repocsv.csv', index= None)

### Connecting multiple topics URL & scraping data segregating in different 

In [40]:
resp= requests.get('https://github.com/topics/3d', headers= header)

In [41]:
for link in Topic_link:
    Resp = requests.get(link, headers= header) 

In [55]:
SOUPELEMT=BeautifulSoup(Resp.content, 'html.parser')

In [ ]:
def get_repo_info(h1_tag, star_tag):
    # 
    

In [79]:
REPOS= SOUPELEMT.find_all('h3', {'class': 'f3 color-fg-muted text-normal lh-condensed'})


for repo in REPOS:
    a= repo.find_all('a')
    b= repo.find_all('a')
    UR_NAME= a[0].text.strip()
    REPO_NAME= b[1].text.strip()
    print(UR_NAME)
    print(REPO_NAME)
     

### Connecting to MySQL database

In [48]:
# Python program to connect
# to mysql database

import mysql.connector as connector

In [49]:
# Connecting from the MySQL server
mydb = connector.connect(
        user= "root",
        host = "localhost",
        passwd = "12345"
)

In [50]:
cur= mydb.cursor()

In [53]:
cur.execute("create database Github_Repo")

In [19]:
create_table= 'create table Topic_link'